### Pandas06 ; NA

In [6]:
import numpy as np
import pandas as pd

### 결측지 처리
- pandas의 기본 틀인 numpy는 부동 소수점에 해당하는 값에만 사용가능한 np.nan이 존재
- 이외의 타입에는 NA값 표기법이 기본적으로 존재하지 않음
- 이외 타입에 해당하는 NA값을 위해 파이썬 내장 객체인 None을 이용=> None을 사용하게 되면 자동으로 dtype = 'object'가 된다

In [11]:
np.array([1,None,2,3]).sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [10]:
np.array([1,np.nan,2,3]).sum()

nan

### 호환
####  pandas가 알아서 처리해준다! => numpy는 그렇지 않음
- 특히 정수일때, NA값 존재시 자동으로 float으로 형변환된다
- 또한 부동 소수점 형태라면 np.nan이든 None이든 다 np.nan을 사용가능

In [23]:
pd.Series([1.1,None,2.333,np.nan,3.222])

0    1.100
1      NaN
2    2.333
3      NaN
4    3.222
dtype: float64

In [12]:
pd.Series([1,None,2,np.nan,3])

0    1.0
1    NaN
2    2.0
3    NaN
4    3.0
dtype: float64

In [5]:
# 계산도 무시하고 알아서잘 해줌
pd.Series([1,None,2,np.nan,3]).mean()

2.0

#### pd.Series 는 여러 데이터 타입이 들어갈수 있음을 명시하자
- 아래의 두 경우 다 가능하다

In [21]:
pd.Series(['1','2',None])

0       1
1       2
2    None
dtype: object

In [20]:
pd.Series(['1','2',np.nan])

0      1
1      2
2    NaN
dtype: object

### 누락된 데이터 탐지
- np.nan None 상관없이 다 탐지한다
- isnull() : bool mask로 return 
- notnull() : bool mask로 return 

In [27]:
pd.Series([1.1,None,2.333,np.nan,3.222]).isnull()

0    False
1     True
2    False
3     True
4    False
dtype: bool

In [28]:
pd.Series(['1','2',np.nan,None]).isnull()

0    False
1    False
2     True
3     True
dtype: bool

In [41]:
# Series 객체 na 존재하는지 확인
pd.Series(['1','2',np.nan,None]).isnull().any()

True

In [43]:
# # Series 객체 na 값 몇개인지 확인
pd.Series(['1','2',np.nan,None]).isnull().sum()

2

In [30]:
s01 = pd.Series(['1','2',np.nan,None])
print('원본데이터')
print(s01)

# NA값 제외하고 출력
print('NA 제외')
print(s01[s01.notnull()])

원본데이터
0       1
1       2
2     NaN
3    None
dtype: object
NA 제외
0    1
1    2
dtype: object


#### data.frame의 경우에는?
- 데이터 프레임의 형식으로 bool mask 반환한다
- any 혹은 sum 함수를 활용하여 더욱 하나의 값으로 반환가능

In [36]:
s01 = pd.Series(['1','2',np.nan,None])
s02 = pd.Series([1,2,3,4])
df01 = pd.DataFrame({'col01':s01,
              'col02':s02})
display(df01)
df01.isnull()

,col01,col02
0,1,1
1,2,2
2,NaN,3
3,None,4


,col01,col02
0,False,False
1,False,False
2,True,False
3,True,False


In [39]:
# 각 열별(axis=0) na값 존재하는지
df01.isnull().any()

col01     True
col02    False
dtype: bool

In [40]:
# 모든 열에(axis=0) na값 존재하는지
df01.isnull().any().any()

True

### 누락된 데이터 처리
- dropna
    - pd.Series 의 경우에는 단순하다 그냥 버린다
    - pd.DataFrae 의 경우에는 행을 지울지 열을 지울지 axis옵션 존재

In [44]:
# series의 경우 dropna
pd.Series(['1','2',np.nan,None]).dropna()

0    1
1    2
dtype: object

In [48]:
# dataframe의 경우 dropna
s01 = pd.Series(['1','2',np.nan,None])
s02 = pd.Series([1,2,3,4])
df01 = pd.DataFrame({'col01':s01,
              'col02':s02})
display(df01)

# NA포함된 행 삭제
display(df01.dropna())

# NA포함된 열 삭제
display(df01.dropna(axis=1))

,col01,col02
0,1,1
1,2,2
2,NaN,3
3,None,4


,col01,col02
0,1,1
1,2,2


,col02
0,1
1,2
2,3
3,4


-  추가적인 dropna option
- how
    - how='any' 널값이 하나라도 있으면 
    - how='all' 널값이 axis에 해당하는 방향에 모두 있어야지
    
- thresh
    - drop 할때 널값이 아닌값이 최소 thresh이상이어야지 살려둔다

In [49]:
s01 = pd.Series(['1','2',np.nan,None])
s02 = pd.Series([1,2,np.nan,4])
df01 = pd.DataFrame({'col01':s01,
              'col02':s02})
display(df01)


,col01,col02
0,1,1.0
1,2,2.0
2,NaN,NaN
3,None,4.0


In [50]:
# nothing changed
df01.dropna(axis=1,how='all')

,col01,col02
0,1,1.0
1,2,2.0
2,NaN,NaN
3,None,4.0


In [52]:
# 행을 따라서 na값이 다 있을때 지운다
df01.dropna(axis=0,how='all')

,col01,col02
0,1,1.0
1,2,2.0
3,None,4.0


In [54]:
# 널값 아닌게 1개이상일경우 살려둔다
df01.dropna(axis=0,thresh=1)

,col01,col02
0,1,1.0
1,2,2.0
3,None,4.0


### 누락 데이터 채우기 
- fillna 
    - 단일 값으로 na 채우기 
    - axis에 따라 앞의 값으로, 뒤의 값으로 na 채우기

In [55]:
s01 = pd.Series(['1','2',np.nan,None])
s02 = pd.Series([1,2,np.nan,4])
df01 = pd.DataFrame({'col01':s01,
              'col02':s02})
display(df01)

,col01,col02
0,1,1.0
1,2,2.0
2,NaN,NaN
3,None,4.0


In [56]:
df01.fillna(1000)

,col01,col02
0,1,1.0
1,2,2.0
2,1000,1000.0
3,1000,4.0


In [57]:
# 앞의 값으로 채우기
df01.fillna(method='ffill')

,col01,col02
0,1,1.0
1,2,2.0
2,2,2.0
3,2,4.0


In [58]:
# 뒤에 값으로 채우기
# 뒤에 값이 없는 경우 안채워질수도 있네
df01.fillna(method='bfill')

,col01,col02
0,1,1.0
1,2,2.0
2,None,4.0
3,None,4.0


In [61]:
df01.fillna(method='bfill',axis=1)

,col01,col02
0,1,1.0
1,2,2.0
2,NaN,NaN
3,4.0,4.0


### 임의로 NA값을 만들어보자
- np.random.choice로 인덱스만 가져오기
- df.sample(frac=)

In [8]:
df01 = pd.DataFrame(np.random.randint(1,50,size=(20,10)))
df01


,0,1,2,3,4,5,6,7,8,9
0,45,13,38,45,43,11,2,26,4,23
1,34,29,34,19,16,23,42,28,22,40
2,1,12,11,38,5,44,40,44,5,21
3,29,45,17,42,28,23,17,22,19,23
4,2,42,15,21,15,31,30,11,40,16
5,15,39,35,17,2,16,30,49,48,47
6,10,3,35,26,42,4,46,43,40,42
7,30,45,24,20,6,9,1,33,20,20
8,27,24,39,28,43,35,13,43,11,33
9,3,2,19,1,27,1,13,13,4,26


In [17]:
# np.random.choice를 사용
df01[0].loc[np.random.choice(df01.index,10)] = np.nan
df01

,0,1,2,3,4,5,6,7,8,9
0,45.0,13,38,45,43,11,2,26,4,23
1,NaN,29,34,19,16,23,42,28,22,40
2,NaN,12,11,38,5,44,40,44,5,21
3,NaN,45,17,42,28,23,17,22,19,23
4,NaN,42,15,21,15,31,30,11,40,16
5,15.0,39,35,17,2,16,30,49,48,47
6,NaN,3,35,26,42,4,46,43,40,42
7,30.0,45,24,20,6,9,1,33,20,20
8,NaN,24,39,28,43,35,13,43,11,33
9,3.0,2,19,1,27,1,13,13,4,26


In [18]:
# df.sample(frac=)사용
df01[3].loc[df01.sample(frac=0.4).index] = np.nan
df01

/Users/iganghui/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,0,1,2,3,4,5,6,7,8,9
0,45.0,13,38,45.0,43,11,2,26,4,23
1,NaN,29,34,NaN,16,23,42,28,22,40
2,NaN,12,11,38.0,5,44,40,44,5,21
3,NaN,45,17,42.0,28,23,17,22,19,23
4,NaN,42,15,21.0,15,31,30,11,40,16
5,15.0,39,35,NaN,2,16,30,49,48,47
6,NaN,3,35,NaN,42,4,46,43,40,42
7,30.0,45,24,20.0,6,9,1,33,20,20
8,NaN,24,39,NaN,43,35,13,43,11,33
9,3.0,2,19,1.0,27,1,13,13,4,26
